In [2]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

print(tf.__version__)

2.2.0


In [3]:

import pandas as pd
df = pd.read_csv ('Dataset1.csv')
print (df)

                                        INTENT  ID Unnamed: 2
0                       Undo the last sentence   1        NaN
1                           Undo the last word   1        NaN
2               Can you undo the last sentence   1        NaN
3                         Please undo the text   1        NaN
4                       Undo the selected text   1        NaN
..                                         ...  ..        ...
423             Please delete all text in bold  26        NaN
424             Kindly delete all text in bold  26        NaN
425                   Erase the last paragraph  26        NaN
426  Erase the last two words of that sentence  26        NaN
427                      Please erase all text  26        NaN

[428 rows x 3 columns]


In [4]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [5]:
label={'1':'UNDO','2':'BOLD','3':'REMOVE_BOLD','4':'ITALIC','5':'REMOVE_ITALIC','6':'UNDERLINE','7':'REMOVE_UNDERLINE','8':'SUPERSCRIPT','9':'REMOVE_SUPERSCRIPT','10':'SUBSCRIPT','11':'REMOVE_SUBSCRIPT','12':'STRIKETHROUGH','13':'REMOVE_STRIKETHROUGH','14':'DELETE','15':'REMOVE_FORMATTING','16':'INSERT_COMMENT','17':'ALIGN_LEFT','18':'ALIGN_RIGHT','19':'ALIGN_CENTER','20':'INSERT_BULLET','21':'NEXT_BULLET','22':'END_BULLET','23':'PAUSE_DICTATION','24':'STOP_DICTATION','25':'SHOW_ALL_COMMANDS','26':'SHOW_HELP'}

In [6]:
df_shuffled.ID.value_counts()

6     26
26    25
18    23
14    22
13    21
24    21
23    20
17    19
2     19
1     18
22    18
3     17
4     17
15    15
5     15
19    14
16    14
21    13
12    13
25    13
9     12
20    12
8     11
7     10
10    10
11    10
Name: ID, dtype: int64

In [7]:
from plotly import __version__
df_shuffled['ID'].value_counts().sort_values(ascending=False).plot(kind='bar', title='Intents for every command')

In [8]:
#Preprocessing text.

In [9]:

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
for index, row in df_shuffled.iterrows():

    tts_string=row['INTENT']
    tts_string=tts_string.lower() #lower case

    #lemmatize, then remove stopwords
    lemmatizer=WordNetLemmatizer()
    lemmatised_string=word_tokenize(tts_string)
    tts_string=""
    for word in lemmatised_string:
        tts_string=tts_string+lemmatizer.lemmatize(word)+" "
    tts_string=tts_string.strip()
    # print(tts_string)

    #remove stopwords
    tokens = word_tokenize(tts_string)
    result = [i for i in tokens if not i in STOPWORDS]
    
    
    custom_stopwords = ['please','could','kindly']
    # using list comprehension to perform task 
    result = [i for i in result if i not in custom_stopwords] 


    # print(result)
    preprocessed_string=""
    preprocessed_string= ' '.join(map(str, result)) 
    print(preprocessed_string)
    # row['INTENT']=preprocessed_string

    #remove certain words
    #lemmatization
    #convert digits to numerals
    #text canonicalization?

    df_shuffled.at[index,'INTENT']=preprocessed_string
   

remove first two character subscript
align line left
insert bullet line page
unbold selected text
end bullet
right align table
remove bold selected text
move next bullet
underline subscript
remove formatting
terminate bulleted list
underline selected word
insert bullet
start next bullet
remove subscript
delete text underlined
undo strikethrough last two character
underline selected word
start inserting bullet selected paragraph
underline selected text
end dictation
add comment
delete text bold
centre align
click cursor position remove superscript
remove emphasis last sentence
underline last word
remove strikethrough
take break dictation
stop dictation near underlined text
position middle
clear italic last two word
decrease subscript fontsize
stop dictation near underlined text
insert bullet
italicize last sentence
present command
insert comment
position image middle page
remove formatting selected sentence
remove last word formatting
shift text right
pause dictation
halt dictation
paus

In [10]:
print(df_shuffled)
#is stemming necessary?


                                   INTENT  ID Unnamed: 2
0    remove first two character subscript  11        NaN
1                         align line left  16        NaN
2                 insert bullet line page  19        NaN
3                    unbold selected text   3        NaN
4                              end bullet  21        NaN
..                                    ...  ..        ...
423               shift paragraph middlwe  16        NaN
424        format emphasize selected text   2        NaN
425                      unbold last word   3        NaN
426        delete last two line paragraph  26        NaN
427            type character superscript   8        NaN

[428 rows x 3 columns]


In [11]:
#Initializing the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_shuffled['INTENT'].values)
word_index = tokenizer.word_index

#maximum number of words in command after preprocessing
MAX_COMMAND_LENGTH = 6

print('Found %s unique tokens.' % len(word_index))

#Tokenizing the texts
X = tokenizer.texts_to_sequences(df_shuffled['INTENT'].values)
X = pad_sequences(X, maxlen=MAX_COMMAND_LENGTH)
print('Shape of data tensor:', X.shape)


#LSTM Modelling
for index, row in df_shuffled.iterrows():
    command=row['INTENT']

#vectorize the commands


Found 125 unique tokens.
Shape of data tensor: (428, 6)


In [12]:
#Convert categorical labels if any to numerical values
Y = pd.get_dummies(df_shuffled['ID']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (428, 26)


In [13]:
#Split training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.5, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(214, 6) (214, 26)
(214, 6) (214, 26)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

#LSTM modelling
EMBEDDING_DIM = 10
MAX_NB_WORDS=1000
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(26, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 200
batch_size = 40

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/200
5/5 [==============================] - 1s 167ms/step - loss: 3.2579 - accuracy: 0.0469 - val_loss: 3.2547 - val_accuracy: 0.1364
Epoch 2/200
5/5 [==============================] - 0s 35ms/step - loss: 3.2540 - accuracy: 0.1146 - val_loss: 3.2540 - val_accuracy: 0.1364
Epoch 3/200
5/5 [==============================] - 0s 35ms/step - loss: 3.2503 - accuracy: 0.1250 - val_loss: 3.2532 - val_accuracy: 0.0909
Epoch 4/200
5/5 [==============================] - 0s 40ms/step - loss: 3.2455 - accuracy: 0.1719 - val_loss: 3.2527 - val_accuracy: 0.0909
Epoch 5/200
5/5 [==============================] - 0s 43ms/step - loss: 3.2422 - accuracy: 0.1198 - val_loss: 3.2517 - val_accuracy: 0.0000e+00
Epoch 6/200
5/5 [==============================] - 0s 40ms/step - loss: 3.2368 - accuracy: 0.1198 - val_loss: 3.2502 - val_accuracy: 0.0000e+00
Epoch 7/200
5/5 [==============================] - 0s 43ms/step - loss: 3.2308 - accuracy: 0.1250 - val_loss: 3.2474 - val_accuracy: 0.0000e+00
Epoch 8

Epoch 59/200
5/5 [==============================] - 0s 50ms/step - loss: 1.2053 - accuracy: 0.7396 - val_loss: 1.5427 - val_accuracy: 0.5909
Epoch 60/200
5/5 [==============================] - 0s 41ms/step - loss: 1.1696 - accuracy: 0.7240 - val_loss: 1.4882 - val_accuracy: 0.5455
Epoch 61/200
5/5 [==============================] - 0s 35ms/step - loss: 1.2260 - accuracy: 0.7135 - val_loss: 1.4949 - val_accuracy: 0.5909
Epoch 62/200
5/5 [==============================] - 0s 41ms/step - loss: 1.1342 - accuracy: 0.7396 - val_loss: 1.5179 - val_accuracy: 0.5909
Epoch 63/200
5/5 [==============================] - 0s 37ms/step - loss: 1.1182 - accuracy: 0.7448 - val_loss: 1.4187 - val_accuracy: 0.5455
Epoch 64/200
5/5 [==============================] - 0s 37ms/step - loss: 1.1144 - accuracy: 0.7500 - val_loss: 1.4043 - val_accuracy: 0.5909
Epoch 65/200
5/5 [==============================] - 0s 38ms/step - loss: 1.0349 - accuracy: 0.8229 - val_loss: 1.3878 - val_accuracy: 0.5455
Epoch 66/200


Epoch 117/200
5/5 [==============================] - 0s 33ms/step - loss: 0.5468 - accuracy: 0.8594 - val_loss: 0.6969 - val_accuracy: 0.8182
Epoch 118/200
5/5 [==============================] - 0s 34ms/step - loss: 0.5117 - accuracy: 0.9062 - val_loss: 0.6927 - val_accuracy: 0.8182
Epoch 119/200
5/5 [==============================] - 0s 35ms/step - loss: 0.5013 - accuracy: 0.9115 - val_loss: 0.6686 - val_accuracy: 0.8182
Epoch 120/200
5/5 [==============================] - 0s 39ms/step - loss: 0.4875 - accuracy: 0.9062 - val_loss: 0.7096 - val_accuracy: 0.8182
Epoch 121/200
5/5 [==============================] - 0s 54ms/step - loss: 0.5196 - accuracy: 0.8594 - val_loss: 0.6749 - val_accuracy: 0.8182
Epoch 122/200
5/5 [==============================] - 0s 48ms/step - loss: 0.4957 - accuracy: 0.8854 - val_loss: 0.6492 - val_accuracy: 0.8182
Epoch 123/200
5/5 [==============================] - 0s 52ms/step - loss: 0.4473 - accuracy: 0.8958 - val_loss: 0.6390 - val_accuracy: 0.8182
Epoch 

Epoch 175/200
5/5 [==============================] - 0s 38ms/step - loss: 0.3288 - accuracy: 0.9323 - val_loss: 0.4065 - val_accuracy: 0.8636
Epoch 176/200
5/5 [==============================] - 0s 40ms/step - loss: 0.3439 - accuracy: 0.8958 - val_loss: 0.3828 - val_accuracy: 0.8636
Epoch 177/200
5/5 [==============================] - 0s 36ms/step - loss: 0.3242 - accuracy: 0.9219 - val_loss: 0.4181 - val_accuracy: 0.9091
Epoch 178/200
5/5 [==============================] - 0s 41ms/step - loss: 0.2398 - accuracy: 0.9531 - val_loss: 0.4180 - val_accuracy: 0.9091
Epoch 179/200
5/5 [==============================] - 0s 37ms/step - loss: 0.3451 - accuracy: 0.9375 - val_loss: 0.4063 - val_accuracy: 0.9091
Epoch 180/200
5/5 [==============================] - 0s 45ms/step - loss: 0.3364 - accuracy: 0.9219 - val_loss: 0.4199 - val_accuracy: 0.8636
Epoch 181/200
5/5 [==============================] - 0s 42ms/step - loss: 0.2884 - accuracy: 0.9427 - val_loss: 0.4360 - val_accuracy: 0.8636
Epoch 

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 10)             10000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 6, 10)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 26)                1326      
Total params: 23,526
Trainable params: 23,526
Non-trainable params: 0
_________________________________________________________________


In [47]:
print(df_shuffled)

INTENT  ID Unnamed: 2
0         right align selected paragraph  17        NaN
1                 remove formatting line  18        NaN
2                 remove formatting line  18        NaN
3                    underline last word   6        NaN
4                             add bullet  19        NaN
..                                   ...  ..        ...
423     underline first n word subscript  10        NaN
424                 unitalicize sentence   5        NaN
425          increase subscript fontsize  10        NaN
426          remove italic last sentence   5        NaN
427  remove last two character subscript  11        NaN

[428 rows x 3 columns]


In [15]:
import sys

sys.path

sys.executable

'/usr/bin/python3.6'